In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pandas as pd
from pathlib import Path
import dateutil.parser
from NPw_aux import read_iono_data, process_ionosphere_files, read_EQ_data, process_eq
from datetime import timedelta

In [ ]:
p = Path("/home/carloscano/data/EQ/").glob("*.csv")
all_pd = pd.DataFrame()

valid_colums = ["time", "longitude", "latitude", "depth", "mag", "type"]
for file in p:
    print(file)
    all_pd = pd.concat([all_pd, pd.read_csv(file,  parse_dates=['time'], date_parser= dateutil.parser.parse).loc[:, valid_colums]])
all_pd = all_pd[all_pd['type'] == "earthquake"].drop("type", axis = 1).set_index("time").sort_index()
all_pd.to_csv("2016_2020_EQ.csv")

In [ ]:

df_eq = read_EQ_data("/home/carloscano/data/EQ/")

In [ ]:
df_process = process_eq(df_eq,df_ion.columns, 5, 2000, 1000, 4.5, 1.5)

In [ ]:
len(df_process)

In [ ]:
df_process

In [ ]:
from NPw_aux import prepare_ion_data

In [ ]:
df_ion, df_eq = prepare_ion_data("GRK")

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from aux_function_SR import read_data, get_eq_filtered, SR_SENSORS
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level, save, load
import logging
import pandas as pd
from datetime import datetime, timedelta
import pickle
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast
from dataclasses import dataclass, asdict
from dateutil.relativedelta import *
from NPw_aux import prepare_ion_data, process_ionosphere_files, remove_outliers
import ast

import sys

In [ ]:
set_log_level("ERROR")

delta = timedelta(minutes=30)
# Read SR and EQ data
freq = timedelta(minutes = 30)
df_ion, df_eq = prepare_ion_data("GRK", freq)
remove_outliers(df_ion, 3)

In [87]:
df_regressor = df_ion.reset_index().copy()
df_events = df_eq.copy()
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast

forecast_length = timedelta(hours=24)
question_mark_length = timedelta(hours=24)
# Time to take into account to predict 
historic_lenght =  timedelta(days=5)
training_lenght = timedelta(days=90)
config_npw_d = {
    "forecast_length": forecast_length,
    "question_mark_length": question_mark_length,
    "freq": freq,
    "training_length": training_lenght,
    "historic_lenght": historic_lenght,
    "drop_missing": False,
    "num_hidden_layers": 10,
    "learning_rate": None,
    "d_hidden": 16,
    "yearly_seasonality": True,
    "daily_seasonality" : False,
    "weekly_seasonality" : True,
    "seasonal_mode": "multiplicative",
    "seasonal_reg": 0,
    "multivariate_season": True,
    "multivariate_trend": True,
    "verbose": False,
    "epochs": 100,
    "drop_missing": False,
    "use_gpu": False,
    "event_type": "Non-Binary",
    "event_mode": "multiplicative",
    "normalization": True,
    "trend_regularization": 0,
    "trend_n_changepoint": 10,
    "sparce_AR": None,
    "loss_func": "Huber",
    "growth": "off"
}

config_npw = ConfigNPw(**config_npw_d)
ConfigEQ_d = {
    "dist_start": 1000,
    "dist_delta": 3000,
    "mag_start": 4.5,
    "mag_delta": 2,
    "filter": True,
    "drop": ["arc_cos", "arc_sin"]
}
config_events = ConfigEQ(**ConfigEQ_d)
from NPw_aux import prepare_eq
_, _, df_events_filtered = prepare_eq(df_events,  config_events.dist_start,
    config_events.dist_delta,
    config_events.mag_start,
    config_events.mag_delta,
    config_events.filter,
    None)
print(len(df_events_filtered))
synthetic_events = pd.read_pickle("synthetic.pkl")
#config_path = "config_002.json"
#(config_npw, config_events) = NPw.save_config(config_path)

m1: 4.5 m2: 6.5 d1: 1000 d2: 4000
1779


In [88]:
df_ion.interpolate(method = "from_derivatives")

,larm,noa1,duth
ds,,,
2016-01-01 00:00:00,24.684500,23.128833,22.445167
2016-01-01 00:30:00,24.653833,23.201333,22.443167
2016-01-01 01:00:00,24.547333,23.295667,22.498500
2016-01-01 01:30:00,23.829500,22.600833,21.898333
2016-01-01 02:00:00,21.875833,20.299667,20.014000
...,...,...,...
2020-12-31 21:30:00,NaN,17.975667,17.936500
2020-12-31 22:00:00,NaN,18.568833,18.575167
2020-12-31 22:30:00,NaN,18.949333,19.030167


In [89]:
df_regressor

,ds,larm,noa1,duth
0,2016-01-01 00:00:00,24.684500,23.128833,22.445167
1,2016-01-01 00:30:00,24.653833,23.201333,22.443167
2,2016-01-01 01:00:00,24.547333,23.295667,22.498500
3,2016-01-01 01:30:00,23.829500,22.600833,21.898333
4,2016-01-01 02:00:00,21.875833,20.299667,20.014000
...,...,...,...,...
87691,2020-12-31 21:30:00,NaN,17.975667,17.936500
87692,2020-12-31 22:00:00,NaN,18.568833,18.575167
87693,2020-12-31 22:30:00,NaN,18.949333,19.030167
87694,2020-12-31 23:00:00,NaN,18.937000,18.957833


In [90]:
NPw_o = NPw(config_npw, df_regressor, config_events, df_events, synthetic_events)

m1: 4.5 m2: 6.5 d1: 1000 d2: 4000


In [91]:
hours_offsets = [0]
event_offsets = [None,  -timedelta(hours=24)]
index = 1


# see what happen in 2019-02-01T10:00:00 Nan values at the end
start_day = datetime.fromisoformat("2018-10-01T10:00:00")

start_date = start_day + index * relativedelta(months=+1)

In [92]:
n = 10

uncer_names = ["step" + str(word) for name in range(n) for word in (str(name) + "___low", str(name) + "___high")]
print(uncer_names)
from scipy import stats

['step0___low', 'step0___high', 'step1___low', 'step1___high', 'step2___low', 'step2___high', 'step3___low', 'step3___high', 'step4___low', 'step4___high', 'step5___low', 'step5___high', 'step6___low', 'step6___high', 'step7___low', 'step7___high', 'step8___low', 'step8___high', 'step9___low', 'step9___high']


In [93]:
number_of_folds = 10
NPw_o.get_folds(k = number_of_folds)

In [97]:
NPw_o.process_fold(0)

Finding best initial lr:   0%|          | 0/252 [00:00<?, ?it/s]

0     14.063052
49    10.685246
98    10.382542
dtype: float32


ValueError: too many values to unpack (expected 2)

In [ ]:
for n_fold in range(number_of_folds):
    try:
        NPw_o.process_fold(n_fold)
    except: #
        print("Problem in fold " + str(n_fold))
        

In [ ]:
import pyarrow.feather as feather
feather.write_feather(NPw_o.get_results()[0], 'test')


In [ ]:
8 //3

In [ ]:
df, df_MAE = NPw_o.get_results()

In [ ]:
df

In [ ]:
df_scale = df.apply(lambda x: ((x -x[0])/x[0]), axis = 1)

In [ ]:
df_scale = ((df_scale / df_scale.abs().max().max()) *100).astype(int)

In [ ]:
NPw.print_df(df)

In [ ]:
df_scale.to_csv("binary_ex.csv")

In [ ]:
print(type(df.index))
b = list()
[b.append(item[0]) for item in df.index if item not in b]
b = (pd.DataFrame(b, columns = ["dates"]).drop_duplicates())
b

In [ ]:
def filter_events(df_events, fc_dt, sort_value, low_td, high_td):
    df_events.index = df_events.index + config_npw.forecast_length // 2
    events = df_events.iloc[(df_events.index > fc_dt - low_td) & (df_events.index < fc_dt + high_td)].sort_values("pr", ascending = False).head()
    if events.empty == False:
        return events

In [ ]:
current_events = list()
for fc_dt in b["dates"]:
    current_events.append(filter_events(df_events.copy(), fc_dt, "pr", timedelta(0), timedelta(150)))

In [ ]:
synthetic_events = [x.iloc[0] for x in current_events[2:8]]

In [ ]:
pd.concat(synthetic_events,axis = 1, ignore_index=True).T.to_pickle("synthetic2.pkl")

In [ ]:
df

In [ ]:
df.to_pickle("ion_test_grk.pkl")

In [ ]:
from scipy.stats import zscore
zetascore_table=zscore(df,axis=0)
zetascore_table

In [ ]:
dates = [x[0] for x in df.keys()].unique()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(50))]

In [ ]:
df

In [ ]:
df_events.iloc[(df_events.index > df.keys()[1][0] - timedelta(0)) & (df_events.index < df.keys()[1][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(0)) & (df_events.index < df.keys()[4][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[6][0] - timedelta(0)) & (df_events.index < df.keys()[6][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[10][0] - timedelta(0)) & (df_events.index < df.keys()[10][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
len(NPw_o.list_keys_fold)

In [ ]:
(NPw_o.list_test_RMSE[0].reset_index())

In [ ]:
a = NPw_o.list_test_RMSE[0][0:3].copy()
b = NPw_o.list_test_RMSE[0][3:6].copy()
c = NPw_o.list_test_RMSE[0][3:6].copy()
a.co"]
b.columns = ["Blumns = ["A"]
c.columns = ["C"]
print(a)
print(b)
r1 = pd.concat([a,b, c], axis = 1).copy()
r2 = pd.concat([a,b, c], axis = 1).copy()
r3 = pd.concat([a,b, c], axis = 1).copy()
print(pd.concat([r1,r2,a], axis=1, keys = [1,2,"NOTHING"]))

In [ ]:
from sdv.tabular import GaussianCopula

In [ ]:
model = GaussianCopula()
model.fit(events)

In [ ]:
sample = model.sample(2000)
sample.head()
print(sample.nlargest(1, columns = ["mag"]))
print(sample.nlargest(5, columns = ["dist"]))
example = sample.nlargest(5, columns = ["dist"])
idx = example.index[[True, False, True, False, True]]

In [ ]:
for idx, ex in example.iterrows():
    print(ex)
    for e in ex.index:
        print(e)

In [ ]:
len(df_train)

In [ ]:
df_train = NPw_o.df_train
delta_t = NPw_o.config_npw.forecast_length

In [ ]:
t0 = df_train.iloc[-(NPw_o.n_forecasts)]["ds"]
t1 = t0 + delta_t

In [ ]:
series = pd.date_range(start=t0, end=t1, periods=5)
series.values

In [ ]:
df_test = df_train.copy()

In [ ]:
df_test["ds"].get_loc()

In [ ]:
print(ev.iloc[1])
print(ev.iloc[2])
ev.iloc[2] = ev.iloc[1]
print(ev.iloc[2])

In [ ]:
for index, (df_train, df_test) in enumerate(NPw_o.folds):
    if index == 10:
        print(df_test)


In [ ]:
config_fc = ConfigForecast(
    start_forecast=start_date, offset_event=None
)
metrics= NPw_o.add_forecast(config_fc
)

In [ ]:
metrics[1]

In [ ]:
NPw_o.one_step_test(model, config_fc, df_test)

In [ ]:
future = model.one_step(df_test, n_historic_predictions=24)

In [ ]:
forecast = model.predict(future, decompose=False, raw=True)
forecast_2  = forecast[forecast["ds"] == start_date].T

In [ ]:
forecast_2.columns = df_test_2.columns  + "_pred"
df_test_2.columns =  df_test_2.columns  + "_current"

In [ ]:
forecast_2 = forecast_2.drop(["ds" ,"ID"]).reset_index().drop("index", axis = 1)

In [ ]:
df_test

In [ ]:
pd.pivot(df_test, index="ds", columns="ID", values="y")[-24:]

In [ ]:
df_all = pd.concat([df_test_2.reset_index(), forecast_2], axis = 1).drop("ds", axis = 1)
df_all.columns = df_all.columns.str.split('_', expand=True)
df_all = df_all.swaplevel(axis=1) 
mse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0)
mae = df_all['current'].sub(df_all['pred']).abs().mean(axis=0)
rmse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0).pow(1/2)
results = pd.concat([mse, mae, rmse], axis = 1)
results

In [ ]:
 pd.pivot(df_test, index="ds", columns="ID", values="y")[-(self.n_forecasts):]

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix

df_test_2.apply(mean_squared_error, axis=0)
